# Feedforward Nerual Network ( FNN )

## Kuhn Poker
Kuhn poker is an extremely simplified form of poker.
Kuhn as a simple model zero-sum two-player imperfect-information game. 
In Kuhn poker, the deck includes only three playing cards, in this deck there will be a Ace, King and Queen. 

* Play:
    - One card is dealt to each player, which may place bets similarly to a standard poker. Both player have now the option to either bet or pass.
    - If both players bet or both players pass, the player with the higher card wins, otherwise, the betting player wins.

In [9]:
from kuhn import Kuhn

In [16]:
# Setup
kuhn = Kuhn()
kuhn.new_match("human","bot")

# First round / Pre showdown
hands = kuhn.pre_showdown()

# Our hand
print("Our human card", hands["human"])

# Showdown
winner = kuhn.showdown()
print(f"\nHands human: {hands['human']} and bot: {hands['bot']}", "\nWinner is:", winner)

Our human card A

Hands human: A and bot: Q 
Winner is: human


In [119]:
def correct_choice(hands, kuhn):
    return [1, 0] if kuhn.deck_value[hands["bot"]] >= kuhn.deck_value[hands["human"]] else [0, 1] # [ 0 -> call, 1 -> fold]

def clear_output(output):
    _, index = torch.max(output, dim=0)
    return [1, 0] if index == 1 else [0, 1]

In [32]:
kuhn = Kuhn()
for _ in range(3):
    print("new match")
    kuhn.new_match("human", "bot")
    hands = kuhn.pre_showdown()
    print(hands, correct_choice(hands, kuhn))
    winner = kuhn.showdown()
    print("showdown:", winner)

new match
{'human': 'K', 'bot': 'A'} [1, 0]
showdown: bot
new match
{'human': 'A', 'bot': 'Q'} [0, 1]
showdown: human
new match
{'human': 'Q', 'bot': 'K'} [1, 0]
showdown: bot


### Neural Networks

In [37]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [101]:
class Bad_Card_Out(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Input layer
        self.fc1 = nn.Linear(1, 64) # Inputs 1 card
        
        # Hidden layers
        self.fc2 = nn.Linear(64, 64)
        #self.fc3 = nn.Linear(64, 64)
        
        # Output layer
        self.fc4 = nn.Linear(64, 2) # Output: call or fold -> 2 classes call, fold
        
        self.values = torch.tensor([1.])
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        #x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.softmax(x, dim=0)            

In [134]:
net = Bad_Card_Out()
optimizer = optim.Adam(net.parameters(), lr=0.001)

EPOCHS = 100
kuhn = Kuhn()
for epoch in range(EPOCHS):
    print(f"Epoch {epoch+1}")
    # Game new match setup
    kuhn.new_match("human", "bot")
    hands = kuhn.pre_showdown()
    #print(hands)
    X = torch.tensor([kuhn.deck_value[hands["bot"]]], dtype=torch.float)
    y = torch.tensor(correct_choice(hands, kuhn))
    
    net.zero_grad()
    output = net(X)
    #print(clear_output(output))
    #print(output, X, y)
    
    loss = F.nll_loss(output,  y) # output = predicted result, y exepected value
    loss.backward()
    optimizer.step()
    print("Loss:", loss.item())

Epoch 1
Loss: -0.48758628964424133
Epoch 2
Loss: -0.4854753911495209
Epoch 3
Loss: -0.517977774143219
Epoch 4
Loss: -0.46938061714172363
Epoch 5
Loss: -0.5657216906547546
Epoch 6
Loss: -0.5815710425376892
Epoch 7
Loss: -0.6042491793632507
Epoch 8
Loss: -0.4257749617099762
Epoch 9
Loss: -0.645378589630127
Epoch 10
Loss: -0.598725438117981
Epoch 11
Loss: -0.6863226890563965
Epoch 12
Loss: -0.6317036747932434
Epoch 13
Loss: -0.3501175343990326
Epoch 14
Loss: -0.44886863231658936
Epoch 15
Loss: -0.32868149876594543
Epoch 16
Loss: -0.675935685634613
Epoch 17
Loss: -0.3172149360179901
Epoch 18
Loss: -0.6858905553817749
Epoch 19
Loss: -0.7833994626998901
Epoch 20
Loss: -0.30094167590141296
Epoch 21
Loss: -0.2966863811016083
Epoch 22
Loss: -0.7045567035675049
Epoch 23
Loss: -0.4314899146556854
Epoch 24
Loss: -0.43281152844429016
Epoch 25
Loss: -0.2921794652938843
Epoch 26
Loss: -0.8032044172286987
Epoch 27
Loss: -0.8038800954818726
Epoch 28
Loss: -0.4460366666316986
Epoch 29
Loss: -0.449481517

In [142]:
print(10*"=", "Playing", "="*10)
kuhn = Kuhn()

matches = 50
w, l, d = 0, 0, 0

for _ in range(matches):
    kuhn.new_match("human", "bot")
    hands = kuhn.pre_showdown()
    x = torch.tensor([kuhn.deck_value[hands["bot"]]], dtype=torch.float)
    output = net(x)
    output = clear_output(output)
    
    if output == [ 1, 0 ]: # -> call 
        winner = kuhn.showdown()
        if winner == "bot":
            w += 1
        else:
            l += 1
    else: # -> fold
        winner = "fold"
        d += 1

    print(f"human: {hands['human']}, bot: {hands['bot']} and bot result", winner)

print(10*"=", "Summary", "="*10)
print('win : ',w)
print('draw: ',d)
print('loss: ',l)

========== Playing ==========
human: A, bot: K and bot result human
human: K, bot: Q and bot result fold
human: A, bot: Q and bot result fold
human: A, bot: Q and bot result fold
human: Q, bot: A and bot result bot
human: A, bot: Q and bot result fold
human: K, bot: Q and bot result fold
human: K, bot: A and bot result bot
human: K, bot: A and bot result bot
human: A, bot: Q and bot result fold
human: Q, bot: K and bot result bot
human: A, bot: Q and bot result fold
human: Q, bot: A and bot result bot
human: K, bot: Q and bot result fold
human: K, bot: Q and bot result fold
human: K, bot: Q and bot result fold
human: A, bot: K and bot result human
human: A, bot: Q and bot result fold
human: A, bot: Q and bot result fold
human: K, bot: Q and bot result fold
human: Q, bot: A and bot result bot
human: K, bot: A and bot result bot
human: Q, bot: A and bot result bot
human: Q, bot: A and bot result bot
human: K, bot: A and bot result bot
human: K, bot: Q and bot result fold
human: Q, bot: K